<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Initial analyses

In [ ]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
# import the class
from sklearn.linear_model import LogisticRegression
# Import label encoder 
from sklearn import preprocessing 

import numpy as np
import matplotlib.pyplot as plt

In [ ]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

# CCD4 uses the new minid's, 2-CC3W is old; but I'm getting error with CCD4
configuration_rid= "2-CCD4" # rid I created with my config containing minid for both train and test sets


In [ ]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

# Generate multimodal wide table

In [ ]:
# old method using local files -- NOT recommended
# multimodal_wide_path = "/data/yukim3003/EyeAI_working/Execution_Assets/Multimodal_Analysis/wide_multimodal_full.csv"
# multimodal_wide = pd.read_csv(multimodal_wide_path)
# multimodal_wide

In [ ]:
# TRAIN: configuration_records.bag_paths[0]
wide_train_raw = EA.severity_analysis(configuration_records.bag_paths[0])

In [ ]:
# TEST: configuration_records.bag_paths[1]
# TRAIN: configuration_records.bag_paths[0]
wide_test_raw = EA.severity_analysis(configuration_records.bag_paths[1])

In [ ]:
# testing duplicate patients where one should have been removed
#wide_train_raw.loc[wide_train_raw['RID_Subject']=="2-7KWW"]
#wide_test_raw.loc[wide_test_raw['RID_Subject']=="2-7MJ4"]

In [ ]:
# add age to table
age_path = "/data/yukim3003/EyeAI_working/Execution_Assets/Multimodal_Analysis/multimodal_subject_age.csv"
age_df = pd.read_csv(age_path)
age_df.rename(columns={'RID': 'RID_Subject'}, inplace=True)
wide_train_raw = wide_train_raw.merge(age_df, on='RID_Subject', how='left')
wide_test_raw = wide_test_raw.merge(age_df, on='RID_Subject', how='left')

# Create new table with only more severe eye for each patient

In [ ]:
# old method with bugs:
#1. if eye1 is GS and eye2 is NaN, then eye2 also becomes GS; and similarly if eye1 has CDR 0.9 but eye2 is NaN, then eye2 also gets CDR 0.9. Fixed this by adding skipna=False to first()
#2. if eye1 has an RNFL but eye2 RNFL is NaN, then this method will consider eye1 to be more severe, whereas it's better to move on to assessing MD in that case. Fixing this fixed 11 eyes

#def pick_severe_eye(df):    
#    # Sort by RNFL, HVF, CDR whereby first row is most severe
#    df = df.sort_values(by=['Average_RNFL_Thickness(μm)', 'MD', 'CDR'], ascending=[True, True, False])
#    
#    # Group by subject and get the first row in each group. If all tied, will just pick the first eye - ie the right eye
#    return df.groupby('RID_Subject').first(skipna=False).reset_index() # first computes the first entry of each column within each group, but NaN's dont count as a value; so if one eye has NaN for any random column, then the value for the other eye is transferred to that eye

#wide_train = pick_severe_eye(wide_train_raw)
#wide_test = pick_severe_eye(wide_test_raw)

# the row that made me realize the bug requiring skipna in first() # if I did want to apply the label of any eye to both eyes, this could be useful
# wide_train[wide_train['RID_Subject']=='2-7KVA']

In [ ]:
# current severity rule: prioritize RNFL > HVF > CDR
# if don't want thresholds, just make threshold 0
# just return the first eye if RNFL, MD, CDR all NaN
def pick_severe_eye(df, rnfl_threshold, md_threshold):
    # Sort by 'Average_RNFL_Thickness(μm)', 'MD', and 'CDR' in descending order
    df = df.sort_values(by=['Average_RNFL_Thickness(μm)', 'MD', 'CDR'], ascending=[True, True, False])

    ### 1. if only 1 eye has a label, just pick that eye as more severe eye (for Dr. Song's patients)
    df = df.groupby('RID_Subject').apply(lambda group: group[group['Label'].notna()]).reset_index(drop=True)
    
    # 2. Select the row/eye with most severe value within the thresholds
    def select_row(group):
        max_value = group['Average_RNFL_Thickness(μm)'].min() # min is more severe for RNFL
        within_value_threshold = group[np.abs(group['Average_RNFL_Thickness(μm)'] - max_value) <= rnfl_threshold] # identify eyes within threshold

        if len(within_value_threshold) > 1 or len(within_value_threshold) == 0: # if both eyes "equal" RNFL OR if RNFL is NaN, then try MD
            max_other_column = within_value_threshold['MD'].min() # min is more severe for MD
            within_other_column_threshold = within_value_threshold[np.abs(within_value_threshold['MD'] - max_other_column) <= md_threshold]

            if len(within_other_column_threshold) > 1 or len(within_other_column_threshold) == 0: # if both eyes "equal" MD OR if MD is NaN, then try CDR
                return group.sort_values(by=['CDR'], ascending=[False]).iloc[0] # since i didn't set CDR threshold, this will always pick something (even if NaN)
            else:
                return within_other_column_threshold.iloc[0]
        else:
            return within_value_threshold.iloc[0]
    return df.groupby('RID_Subject').apply(select_row).reset_index(drop=True)

In [ ]:
wide_train_nothresh = pick_severe_eye(wide_train_raw, 0, 0)
wide_test_nothresh = pick_severe_eye(wide_test_raw, 0, 0)

In [ ]:
rnfl_thresh = 0
md_thresh = 0
wide_train = pick_severe_eye(wide_train_raw, rnfl_thresh, md_thresh)
wide_test = pick_severe_eye(wide_test_raw, rnfl_thresh, md_thresh)

In [ ]:
wide_train
wide_train.loc[wide_train['RID_Subject']=="2-7M2E"] # duplicate subject in orig database, same as 2-7NGA
#wide_train.loc[wide_train['RID_Subject']=="2-7KTT"] # example subject where more severe eye Left should be returned
#wide_train.loc[wide_train['RID_Subject']=="2-7KVA"] # example subject where only eye with label Left should be returned

In [ ]:
# Show which subjects changed eyes by adding thresholds
diff_values = wide_train.compare(wide_train_nothresh, align_axis=0, keep_shape=True, keep_equal=True) #keep_equal=False --> values that are equal are represented as NaN
diff_values = diff_values.drop_duplicates(keep=False) # drop rows that have a duplicate
print("# subjects where eye choice changed: %i" % (len(diff_values)/2))
diff_values[['RID_Subject', 'Side', 'Label', 'Average_RNFL_Thickness(μm)', 'MD', 'CDR']]

# Transform Train and Test Data

In [ ]:
#split dataset in features and target variable
demographic_fx = ['Gender', 'Ethnicity', 'Age']
clinic_fx = ['LogMAR_VA', 'IOP', 'CDR'] # 'Gonioscopy' - mostly NaN, not standardized annotation # CCT - mostly NaN
HVF_fx = ['MD', 'VFI'] # 'PSD' - mostly NaN. I think PSD and PSD.1 columns should be merged to use this column if desired
RNFL_fx = ['Average_RNFL_Thickness(μm)'] # Average_C/D_Ratio - for RNFL-derived CDR
RNFL_clockhr_fx = ['Clock_Hours_1', 'Clock_Hours_2', 'Clock_Hours_3', 'Clock_Hours_4', 'Clock_Hours_5', 'Clock_Hours_6', 'Clock_Hours_7', 'Clock_Hours_8', 'Clock_Hours_9', 'Clock_Hours_10', 'Clock_Hours_11', 'Clock_Hours_12'] # if I want to use each clock hour
RNFL_quad_fx = ['Quadrants_S', 'Quadrants_N', 'Quadrants_T', 'Quadrants_I']
RNFL_IS_fx = ['Quadrants_S', 'Quadrants_I']
GHT = ['GHT']

fx_cols = demographic_fx + clinic_fx + HVF_fx + RNFL_fx + RNFL_IS_fx + GHT # selected feature cols from above
# fx_cols = ['CDR', 'MD'] + RNFL_IS_fx + GHT

In [ ]:
# Method to transform data, to apply to wide_train and wide_test
def transform_data(multimodal_wide, fx_cols):
    ### drop rows missing label (ie no label for POAG vs PACG vs GS)
    multimodal_wide = multimodal_wide.dropna(subset=['Label'])
    # drop rows where label is "Other" (should only be PACG, POAG, or GS)
    allowed_labels = ["PACG", "POAG", "GS"]
    multimodal_wide = multimodal_wide[multimodal_wide['Label'].isin(allowed_labels)]

    X = multimodal_wide[fx_cols] # Features
    y = multimodal_wide.Label # Target variable

    ### GHT: reformat as "Outside Normal Limits", "Within Normal Limits", "Borderline", "Other"
    if "GHT" in fx_cols:
        GHT_categories = ["Outside Normal Limits", "Within Normal Limits", "Borderline"]
        X.loc[~X['GHT'].isin(GHT_categories), 'GHT'] = np.nan # alt: 'Other'; I did np.nan bc I feel like it makes more sense to drop this variable

    ### categorical data: encode using LabelEncoder or OneHotEncoder
    # label encoder if data ordinal (ie ranked) -- jk nvm this transformer should be used to encode target values, i.e. y, and not the input X!
    # one-hot if data not ranked; note this will increase dimensionality of data which is bad if >1/3rd of fx are one-hot
    # https://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor
    #one_hot_encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
    from feature_engine.encoding import OneHotEncoder # this instead of skLearn allows me to one hot encode desired columns only
    categorical_vars = list(set(fx_cols) & set(['Gender', 'Ethnicity', 'GHT']))  # cateogorical vars that exist

    if len(categorical_vars)>0: 
        # replace NaN with category "Unknown", then delete this column from one-hot encoding later
        for var in categorical_vars:
            X[var] = X[var].fillna("Unknown")
        
        encoder = OneHotEncoder(variables = categorical_vars)
        X_transformed = encoder.fit_transform(X)

        # delete Unknown columns
        X_transformed.drop(list(X_transformed.filter(regex='Unknown')), axis=1, inplace=True)

        ### sort categorical encoded columns so that they're in alphabetical order
        def sort_cols(X, var):
            # Select the subset of columns to sort
            subset_columns = [col for col in X.columns if col.startswith(var)]
            # Sort the subset of columns alphabetically
            sorted_columns = sorted(subset_columns)
            # Reorder the DataFrame based on the sorted columns
            sorted_df = X[[col for col in X.columns if col not in subset_columns] + sorted_columns]
            return sorted_df
        for var in categorical_vars:
            X_transformed = sort_cols(X_transformed, var)

    else:
        print("No categorical variables")
        X_transformed=X

    ### format numerical data
    # VFI
    if 'VFI' in fx_cols:
        X_transformed['VFI'] = X_transformed['VFI'].replace('Off', np.nan) # replace "Off" with nan
        def convert_percent(x):
            if pd.isnull(x):
                return np.nan
            return float(x.strip('%'))/100
        X_transformed['VFI'] = X_transformed['VFI'].map(convert_percent)

    ### format y
    # combine PACG and POAG as glaucoma
    y = y.replace(['POAG', 'PACG'], 'Glaucoma')
    # convert to 0 and 1
    label_encoder = preprocessing.LabelEncoder()
    y[:] = label_encoder.fit_transform(y) # fit_transform combines fit and transform
    y = y.astype(int)

    return X_transformed, y

In [ ]:
X_train_transformed, y_train_keep_missing = transform_data(wide_train, fx_cols)
X_test_transformed, y_test_keep_missing = transform_data(wide_test, fx_cols)

In [ ]:
### normalize numeric training data (so that features are on same scale instead of wildly different scales)
# not required for typical logistic regression, but do need for regularized regression
# I didn't put this in transform_data because I want to use the scaler fitted on train for test too

# how? https://datascience.stackexchange.com/questions/54908/data-normalization-before-or-after-train-test-split
# why? https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

from sklearn.preprocessing import StandardScaler
categorical_vars = ['Gender', 'Ethnicity', 'GHT']
numeric_vars = sorted(set(fx_cols) - set(categorical_vars), key=fx_cols.index)

scaler = StandardScaler()

normalized_numeric_X_train = pd.DataFrame(
    scaler.fit_transform(X_train_transformed[numeric_vars]),
    columns = numeric_vars
)
cat_df = X_train_transformed.drop(numeric_vars, axis=1)
X_train_keep_missing = pd.concat([normalized_numeric_X_train.set_index(cat_df.index), cat_df], axis=1)

# normalize test data, but using scaler fitted to training data to prevent data leakage
normalized_numeric_X_test = pd.DataFrame(
    scaler.transform(X_test_transformed[numeric_vars]),
    columns = numeric_vars
)
cat_df = X_test_transformed.drop(numeric_vars, axis=1)
X_test_keep_missing = pd.concat([normalized_numeric_X_test.set_index(cat_df.index), cat_df], axis=1)


# Counts / data info

In [ ]:
counts = np.unique(y_train_keep_missing, return_counts=True)
print(counts) # #GS vs #Glaucoma
print("Percent GS vs Glaucoma in TRAIN:", counts[1] / sum(counts[1])) # percent

counts = np.unique(y_test_keep_missing, return_counts=True)
print(counts) # #GS vs #Glaucoma
print("Percent GS vs Glaucoma in TEST:", counts[1] / sum(counts[1])) # percent

In [ ]:
# #NAN
### the number of rows with nan in any column will increase if I choose more features

# count number / percent of rows with nan value
num_rows_with_nan = X_train_keep_missing.isnull().any(axis=1).sum()
print ("Number of train rows with any nan: %i" % num_rows_with_nan)

# Calculate the percentage of rows with NaN values
print ("Percent of train rows with any nan: %f" % ((num_rows_with_nan / len(X_train_keep_missing)) * 100))

# count number / percent of rows with nan value
num_rows_with_nan = X_test_keep_missing.isnull().any(axis=1).sum()
print ("Number of test rows with any nan: %i" % num_rows_with_nan)

# Calculate the percentage of rows with NaN values
print ("Percent of test rows with any nan: %f" % ((num_rows_with_nan / len(X_test_keep_missing)) * 100))

# A) Simple imputation

In [ ]:
strat = 'mean'
# NOTE: the following code imputes X_test based on the imputer fitted to X_train

"""
STRATEGIES
If “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.

If “median”, then replace missing values using the median along each column. Can only be used with numeric data.

If “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.

If “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.
"""

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy=strat)
imputer = imputer.fit(X_train_keep_missing)
X_train_imputed = imputer.transform(X_train_keep_missing)
X_test_imputed = imputer.transform(X_test_keep_missing)
# convert into pandas dataframe instead of np array
X_train = pd.DataFrame(X_train_imputed, columns=X_train_keep_missing.columns)
X_test = pd.DataFrame(X_test_imputed, columns=X_test_keep_missing.columns)

y_train = y_train_keep_missing
y_test = y_test_keep_missing

# B) Multiple imputations (skip this section if not doing imputations)

In [ ]:
# good article on MCAR vs MAR vs MNAR and how to appropriately handle missing values in each case: https://datascience.stackexchange.com/questions/116622/what-should-you-do-with-nan-values

In [ ]:
#def handle_missing(X_transformed):
    ### Handle missing values
    # Xu:
    # - In the past, we’ve used multiple imputation as long as the % of missing values was less than 10% for any given variable. I attached a paper we wrote where we used this technique. 
    # - Balancing can be done by upsampling the minority class, although in this case the two are fairly similar in number."
    # https://scikit-learn.org/stable/modules/impute.html
    
    ### old simple imputation method
    #from sklearn.impute import SimpleImputer
    #imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    #imputer = imputer.fit(X_transformed)
    #X_transformed[:] = imputer.transform(X_transformed) # [:] modifies the df in place

# return list of pandas dataframes, each containing 1 of 10 imputations
def mult_impute_missing(X, train_data=None):
    if train_data is None:
        train_data = X

    ### multiple imputation method
    # IterativeImputer from sklearn is an alternative multivariate simple imputation method, that can be changed to be a multiple imputation method by iterating through multiple random states. However, this 
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer

    imp = IterativeImputer(max_iter=10, random_state=0, sample_posterior=True)

    imputed_datasets = []
    for i in range(10): # 3-10 imputations standard
        imp.random_state = i
        imp.fit(train_data)
        X_imputed = imp.transform(X)
        imputed_datasets.append(pd.DataFrame(X_imputed, columns=X.columns))

    # ALTERNATIVE
    #from statsmodels.imputation import mice.MICEData # alternative package for MICE imputation
    # official docs: https://www.statsmodels.org/dev/generated/statsmodels.imputation.mice.MICE.html#statsmodels.imputation.mice.MICE
    # multiple imputation example using statsmodels: https://github.com/kshedden/mice_workshop
    #imp = mice.MICEData(data)
    #fml = 'y ~ x1 + x2 + x3 + x4' # variables used in multiple imputation model
    #mice = mice.MICE(fml, sm.OLS, imp) # OLS chosen; can change this up
    #results = mice.fit(10, 10) # 10 burn-in cycles to skip, 10 imputations
    #print(results.summary())
    
    return imputed_datasets

In [ ]:
X_train_imputedsets = mult_impute_missing(X_train_keep_missing) # list of 10 imputed X_trains

In [ ]:
X_test_imputedsets = mult_impute_missing(X_test_keep_missing, train_data=X_test_keep_missing) # Impute test data using model fit with training data, not with test data!

# C) Drop NA

In [ ]:
# drop rows with nan
X_train = X_train_keep_missing.dropna()
X_test = X_test_keep_missing.dropna()

y_train = y_train_keep_missing[y_train_keep_missing.index.isin(X_train.index)]
y_test = y_test_keep_missing[y_test_keep_missing.index.isin(X_test.index)]

# Model methods

In [ ]:
from scipy.stats import norm

# print model coefficients, ORs, p-values
def model_summary(model, X_train):
    coefs = model.coef_[0]
    # odd ratios = e^coef
    ors = np.exp(coefs)
    intercept = model.intercept_[0]

    ### 2 ways to calculate p-values; NOTE THAT P VALUES MAY NOT MAKE SENSE FOR REGULARIZED MODELS
    # https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
    def logit_pvalue(model, x):
        """ Calculate z-scores for scikit-learn LogisticRegression.
        parameters:
            model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
            x:     matrix on which the model was fit
        This function uses asymtptics for maximum likelihood estimates.
        """
        p = model.predict_proba(x)
        n = len(p)
        m = len(model.coef_[0]) + 1
        coefs = np.concatenate([model.intercept_, model.coef_[0]])
        x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
        ans = np.zeros((m, m))
        for i in range(n):
            ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
        vcov = np.linalg.inv(np.matrix(ans))
        se = np.sqrt(np.diag(vcov))
        t =  coefs/se  
        p = (1 - norm.cdf(abs(t))) * 2
        return p
    p_values = logit_pvalue(model, X_train)
    def format_pvalue(p_values):
        f = ["<.001" if x<0.001 else "%.3f"%x for x in p_values]
        return f
    #print(format_pvalue(pvalues))
    
    # compare with statsmodels ### RESULT: produces same result except gives nan instead of 1.00 for insignficant p-values
    #import statsmodels.api as sm
    #sm_model = sm.Logit(y_train, sm.add_constant(X_train)).fit(disp=0) ### these uses y_train from outside this function so not really valid but oh well I just want it for testing purposes
    #p_values=sm_model.pvalues
    #print(format_pvalue(pvalues))
    #sm_model.summary()

    # print results
    results = pd.DataFrame({
        'Coefficient': np.append(intercept, coefs),
        'Odds Ratio': np.append(np.exp(intercept), ors),
        'P-value': format_pvalue(p_values)
    }, index=['Intercept'] + list(X_train.columns))
    print(results)

In [ ]:
# model performance
# https://medium.com/javarevisited/evaluating-the-logistic-regression-ae2decf42d61

print('Training set count: %i' % len(X_train))
print('Test set count: %i' % len(X_test))

def compute_performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    import sklearn.metrics as metrics
    # evaluate predictions
    mae = metrics.mean_absolute_error(y_test, y_pred)
    print('MAE: %.3f' % mae)
    
    # examine the class distribution of the testing set (using a Pandas Series method)
    y_test.value_counts()
    
    # calculate the percentage of ones
    # because y_test only contains ones and zeros, we can simply calculate the mean = percentage of ones
    y_test.mean()
    
    # calculate the percentage of zeros
    1 - y_test.mean()
    
    
    # # Metrics computed from a confusion matrix (before thresholding)
    
    # Confusion matrix is used to evaluate the correctness of a classification model
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(y_test,y_pred)
    confusion_matrix
    
    TP = confusion_matrix[1, 1]
    TN = confusion_matrix[0, 0]
    FP = confusion_matrix[0, 1]
    FN = confusion_matrix[1, 0]
    
    # Classification Accuracy: Overall, how often is the classifier correct?
    # use float to perform true division, not integer division
    # print((TP + TN) / sum(map(sum, confusion_matrix))) -- this is is the same as the below automatic method
    print('Accuracy: %.3f' % metrics.accuracy_score(y_test, y_pred))
    
    # Sensitivity(recall): When the actual value is positive, how often is the prediction correct?
    sensitivity = TP / float(FN + TP)
    
    print('Sensitivity: %.3f' % sensitivity)
    # print('Recall score: %.3f' % metrics.recall_score(y_test, y_pred)) # same thing as sensitivity, but recall term used in ML
    
    # Specificity: When the actual value is negative, how often is the prediction correct?
    specificity = TN / (TN + FP)
    print('Specificity: %.3f' % specificity)
    
    #from imblearn.metrics import specificity_score
    #specificity_score(y_test, y_pred)

    # False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
    false_positive_rate = FP / float(TN + FP)
    print('FPR: %.3f' % false_positive_rate)
    # print(1 - specificity) # same as FPR
    
    # Precision: When a positive value is predicted, how often is the prediction correct?
    precision = TP / float(TP + FP)
    #print('Precision: %.3f' % precision)
    print('Precision: %.3f' % metrics.precision_score(y_test, y_pred))
    
    # F score
    f_score = 2*TP / (2*TP + FP + FN)
    #print('F score: %.3f' % f_score)
    print('F1 score: %.3f' % metrics.f1_score(y_test,y_pred))
    
    #Evaluate the model using other performance metrics - A BIT REDUNDANT, COMMENTED OUT FOR NOW
    # from sklearn.metrics import classification_report
    # print(classification_report(y_test,y_pred))

    # AUC
    y_pred_proba = model.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    print('AUC: %.3f' % auc)

    # CM matrix plot
    from sklearn import metrics
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = None)
    
    cm_display.plot()
    plt.show()
    # 0 = GS, 1 = POAG
    
    # ROC curve plot
    plt.plot(fpr,tpr,label="auc="+str(auc))
    plt.xlabel("False positive rate (1-specificity)")
    plt.ylabel("True positive rate (sensitivity)")
    plt.legend(loc=4)
    plt.show()

# Logistic Regression DROPNA or SIMPLEIMPUTER

In [ ]:
penalty=None#'l1', 'l2', 'elasticnet', or None
solver='saga' # 'lbfgs', 'saga' (only saga supports l1 and elasticnet)

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16, solver=solver, max_iter=1000, penalty=penalty)

# fit the model with data
logreg.fit(X_train, y_train)
model_summary(logreg, X_train)
compute_performance(logreg, X_test, y_test)

In [ ]:
# using statsmodels instead -- IGNORE THIS CELL, I FIGURED OUT THAT P-VALUES AREN'T REALLY APPROPRIATE FOR REGULARIZED REGRESSION --> JUST USE SKLEARN PACKAGE
"""
import statsmodels.api as sm

# Adding a constant term for the intercept
X = sm.add_constant(X_train)
model = sm.Logit(list(y_train), X)

# normal logistic regression
result = model.fit()
result.summary()

# Lasso (L1) logistic regression
# i think the statsmodels fit_regularized method for logistic regression only has the lasso method?
lasso_logit = model.fit_regularized(method='l1', alpha=0.1)
lasso_logit.summary()
                                    
# Elastic Net (L1 and L2) logistic regression
# has to be implemented using GLM, whereby choosing binomial family is essentially logistic regression
# model = sm.GLM(list(y_train), X, family=sm.families.Binomial())
# elastic_net_logit = model.fit_regularized(method='elastic_net', alpha=0.1) # https://www.statsmodels.org/devel/generated/statsmodels.genmod.generalized_linear_model.GLM.fit_regularized.html
#print(dir(elastic_net_logit))
#elastic_net_logit.params
#elastic_net_logit.summary() # this is not implemented
"""


In [ ]:
#### Regularization params
k_folds = 10 #5-10 standard
scoring = 'roc_auc' # 'accuracy' (default), 'roc_auc', 'neg_mean_absolute_error' ...options on sklearn.metrics: https://scikit-learn.org/stable/api/sklearn.metrics.html#module-sklearn.metrics
max_iter=1000
solver='saga'
# for elastic net only:
lambda_inverse = 100  # of C's (=inverse of lambda) to try; 10 by default
alpha_range = np.linspace(0, 1, 100)

from sklearn.linear_model import LogisticRegressionCV

In [ ]:
# 1) Ridge
ridge_cv = LogisticRegressionCV(cv=k_folds, scoring=scoring, solver=solver, max_iter=max_iter)
ridge_cv.fit(X_train, y_train)
# Retrieve the best hyperparameters
best_C = ridge_cv.C_[0]
print(f"Best C (inverse of regularization strength): {best_C}")

model_summary(ridge_cv, X_train)
compute_performance(ridge_cv, X_test, y_test)

In [ ]:
# 2) Elastic Net
#https://stackoverflow.com/questions/66787845/how-to-perform-elastic-net-for-a-classification-problem
# SAGA should be considered more advanced and used over SAG. For more information, see: https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions
en_cv = LogisticRegressionCV(cv=k_folds, scoring=scoring, penalty='elasticnet', Cs = lambda_inverse, l1_ratios=alpha_range, solver=solver, max_iter=max_iter)
en_cv.fit(X_train, y_train)

# Retrieve the best hyperparameters
best_C = en_cv.C_[0]
best_l1_ratio = en_cv.l1_ratio_[0]
print(f"Best C (inverse of regularization strength): {best_C}")
print(f"Best l1_ratio (mixing parameter): {best_l1_ratio}")

In [ ]:
print(f"Best C (inverse of regularization strength): {best_C}")
print(f"Best l1_ratio (mixing parameter): {best_l1_ratio}")
model_summary(en_cv, X_train)
compute_performance(en_cv, X_test, y_test)

# Logistic Regression MULTIPLE IMPUTATIONS

In [ ]:
# how to do prediction after multiple imputation:
# https://github.com/amices/mice/issues/82
# https://stackoverflow.com/questions/68460923/how-to-do-the-prediction-after-multiple-imputation-with-mice-package
logreg_models = []
ypred_results = []

for X_train in X_train_imputedsets:
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression(random_state=16, solver='lbfgs', max_iter=1000)
    logreg.fit(X_train, y_train)
    logreg_models.append(logreg)

    for X_test in X_test_imputedsets:
        y_pred = logreg.predict(X_test)
        ypred_results.append(y_pred)

In [ ]:
# just took the mode for predictions from my 10 imputations, each fitted on 1 of 10 train imputations
# figure out if there's a better method for this
from scipy.stats import mode
y_pred = mode(ypred_results, axis=0).mode

# does the same thing:
#ypred_df = pd.DataFrame(np.row_stack(ypred_results)) # 10x10 df
#y_pred = ypred_df.mode(axis=0).loc[0].astype(int) # get mode of each column
#np.array(y_pred)

In [ ]:
### After performing logistic regression on each imputed dataset, pool the results using Rubin’s rules to obtain a single set of estimates.

# Extract coefficients and standard errors
coefs = np.array([model.coef_[0] for model in logreg_models])
intercepts = np.array([model.intercept_[0] for model in logreg_models])

# Calculate pooled estimates
pooled_coefs = np.mean(coefs, axis=0)
pooled_intercept = np.mean(intercepts)
# I think this calculates SES between the imputed datasets
pooled_ses = np.sqrt(np.mean(coefs**2, axis=0) + np.var(coefs, axis=0, ddof=1) * (1 + 1/len(logreg_models)))

# Display pooled results
results = pd.DataFrame({
    'Coefficient': np.append(pooled_intercept, pooled_coefs),
    'Standard Error': np.append(np.nan, pooled_ses)  # Intercept SE is not calculated here
}, index=['Intercept'] + list(X_train.columns))
print(results)

# template stuff I haven't deleted

In [ ]:
# View data

# subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv')
# subject

# observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv')
# observation

# clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv')
# clinic

# observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv')
# observation_clinic_asso # association table between observation table and clinic record table

# icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv')
# icd10

# icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv')
# icd10_asso # association table between clinic record table and ICD10 code

# report = pd.read_csv(configuration_records.bag_paths[0]/'data/Report.csv')
# report

# RNFL_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/RNFL_OCR.csv')
# RNFL_OCR

HVF_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/HVF_OCR.csv')
HVF_OCR


In [ ]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_hyper_parameter_tuning import main #import the new logistic module.
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main()


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

# 